In [1]:
import cv2
import mediapipe as mp
import os
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import modules.mod_faceDetection as fd
import sys
import inspect

In [2]:
# import modules from sibling directory "gender"
currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir +  os.path.sep + 'gender') # append gender directory to python path to allow module reading

from gender_config import *

In [3]:
def get_face_box(bbox_dict, frame):
    x_min, y_min, w, h = bbox_dict.values()
    frame_h, frame_w, _ = frame.shape
    # get centerpoint
    center_w = x_min + w/2
    center_h = y_min + h/2
    centerpoint = int(center_w * frame_w), int(center_h * frame_h)
    # square size is max of width/length based on mediapipe bounding box
    face_size = max(w * frame_w, h * frame_h)
    # face_size = int(face_size + face_size*0.1) # extend the face detection box by 10%
    face_size = int(face_size)

    # get the 4 corners of face, or clip to edge of image
    face_size_half = int(face_size/2) # since we start with centerpoint

    left = np.clip(centerpoint[0] - face_size_half, a_min=0, a_max=frame_w)
    right = np.clip(centerpoint[0] + face_size_half, a_min=0, a_max=frame_w)
    bottom = np.clip(centerpoint[1] + face_size_half, a_min = 0, a_max = frame_h)
    top = np.clip(centerpoint[1] - face_size_half, a_min = 0, a_max = frame_h)

    tl = (left, top)
    bl = (left, bottom)
    tr = (right, top)
    br = (right, bottom)

    # # Note that colors below are "incorrect" because opencv uses BGR
    # cv2.circle(frame, tl, radius = 2, color = (0,0,255), thickness = 2)
    # cv2.circle(frame, bl, radius = 2, color = (0,255,0), thickness = 2)
    # cv2.circle(frame, tr, radius = 2, color = (255,0,0), thickness = 2)
    # cv2.circle(frame, br, radius = 2, color = (0,255,255), thickness = 2)

    # return topleft point and face_size, for use with cropping
    return left, right, bottom, top

In [4]:
def crop_face(left, right, bottom, top, frame):
    return frame[top:bottom, left:right, :]

### Starts webcam thing

In [5]:
# MAIN

# load in our models
# model paths
GENDER_MODEL_PATH = '../gender/gender_model.hdf5'
# get models
gender_model = keras.models.load_model(GENDER_MODEL_PATH)

# Initiate face detection model
face_detector = fd.FaceDetection(model_selection = 0, threshold = 0.7)

# param is 0, first webcam in list of webcams
cam = cv2.VideoCapture(0)

while cam.isOpened():
    # take frame from webcam
    success, frame = cam.read()
    # flip frame for selfie mode
    frame = cv2.flip(frame, 1)
    # check if loaded frame
    if not success:
        print("Image is donezo gonezo")
        continue

    # frame setup
    frame.flags.writeable = False # not writeable, pass by reference, makes it faster
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # rgb for mediapipe use


    # detect boxes
    bboxs = face_detector.get_bboxs(frame)

    # draw bbox on frame
    frame.flags.writeable = True # now need to draw on
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR) # back to bgr for opencv

    if bboxs:
        for bbox in bboxs:
            # crop face for models
            face_left, face_right, face_bottom, face_top = get_face_box(bbox[1], frame) # obtain lrbt for slicing the image
            face_cropped = crop_face(face_left, face_right, face_bottom, face_top, frame) # crops the face
            face_cropped = cv2.resize(face_cropped, (200, 200))

            # predicts with models
            gender_score = gender_model.predict(np.array([face_cropped]))[0][0]
            gender = "F" if gender_score >= 0.5 else "M"

            # write gender
            face_detector.draw_bbox(bbox[0], bbox[1], frame, gender = gender, gender_score = gender_score)

            #Draw box on face
            # face_detector.draw_bbox(bbox[0], bbox[1], frame)

    # display frame
    cv2.imshow('Webcam', frame)

    # use q to quit
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break


# close the camera
cam.release()
# Close windows
cv2.destroyAllWindows()